# Select Images for Composite Variations
## TODO:
- Split multipolygon in Nitze BBoxes
- Right join IDs from ImageCollections to dataframe with footprints and bbox IDs
- Do I actually need to read in the BBoxes separately, or can I use the info from the ImageCollection IDs?

## Set Up Environment

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import re
from google.cloud import storage
from natsort import natsorted

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Define Functions

In [ ]:
# apply image IDs to each image in the collection
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;


## Import Data and Prepare Visualization Parameters

In [ ]:
# Import Planet Data GCS
train = ee.ImageCollection('projects/abruptthawmapping/assets/yg_train_regions_imagery_calibrated')
train = train.map(setID)
val = ee.ImageCollection('projects/abruptthawmapping/assets/yg_val_regions_imagery_calibrated')
val = val.map(setID)
nitze = ee.ImageCollection('projects/abruptthawmapping/assets/nitze_regions_imagery_calibrated')
nitze = nitze.map(setID)


In [ ]:
# Get Image IDs and BBox IDs from ImageCollection IDs
yg_train_images = pd.DataFrame({'img_id': natsorted(train.aggregate_array('ID').getInfo())})
yg_train_images['bbox_id'] = [re.split('_\d{8}_', img_id)[0] for img_id in yg_train_images.img_id]
yg_train_images['img_id'] = [
    re.split(
        '_3B.+_mad$',
        re.split('^\d+_', 
                 img_id)[1]
    )[0] 
    for img_id in yg_train_images.img_id]

yg_val_images = pd.DataFrame({'img_id': natsorted(val.aggregate_array('ID').getInfo())})
yg_val_images['bbox_id'] = [re.split('_\d{8}_', img_id)[0] for img_id in yg_val_images.img_id]
yg_val_images['img_id'] = [
    re.split(
        '_3B.+_mad$',
        re.split('^\d+_', 
                 img_id)[1]
    )[0] 
    for img_id in yg_val_images.img_id]

nitze_images = pd.DataFrame({'img_id': natsorted(nitze.aggregate_array('ID').getInfo())})
nitze_images['bbox_id'] = [re.split('_\d{8}_', img_id)[0] for img_id in nitze_images.img_id]
nitze_images['img_id'] = [
    re.split(
        '_3B.+_mad$',
        re.split('^\d+_', 
                 img_id)[1]
    )[0] 
    for img_id in nitze_images.img_id]

In [ ]:
yg_train_images.iloc[-2:-1]

In [ ]:
yg_val_images.iloc[-2:-1]

In [ ]:
nitze_images.iloc[-2:-1]

In [ ]:
# Import Bounding Boxes
yg_train_bboxes = gpd.read_file("../../data/yg_train_regions/bboxes/RTS_buffer_separate.shp")
yg_val_bboxes = gpd.read_file("../../data/yg_val_regions/bboxes/yg_validation_bboxes.shp")
nitze_bboxes = gpd.read_file("../../data/nitze_regions/bboxes/nitze_bbox_water_removed.shp")

In [ ]:
nitze_bboxes

In [ ]:
# Import Image Footprints
dir_path = ("../../data/yg_val_regions/image_footprints/")
val_footprint_paths = []
for root, subdirs, files in os.walk(dir_path):
    for file in files:
        if re.match('.*shp$', file):
            val_footprint_paths.append(os.path.join(root, file))
pprint(val_footprint_paths)

yg_val_footprints = gpd.read_file(val_footprint_paths[0])

dir_path = ("../../data/nitze_regions/image_footprints/")
files = storage_client.list_blobs("abrupt_thaw", prefix = dir_path)
nitze_footprint_paths = []
for root, subdirs, files in os.walk(dir_path):
    for file in files:
        if re.match('.*[^(old)].shp$', file):
            nitze_footprint_paths.append(os.path.join(root, file))
pprint(nitze_footprint_paths)

nitze_footprints = gpd.GeoDataFrame()
for file in nitze_footprint_paths:
    data = gpd.read_file(file)
    nitze_footprints = pd.concat([nitze_footprints, data], ignore_index = True)


In [ ]:
yg_val_footprints

In [ ]:
nitze_footprints

In [ ]:
# View the imagery
vis_params_imagery = {
    'min': 0,'max': 1200,
    'bands': ['red', 'green', 'blue'],
    'gamma': 0.9
}

## Select Images

In [ ]:
# Join Footprints with bboxes
yg_val_footprints = yg_val_footprints.sjoin(yg_val_bboxes.to_crs(yg_val_footprints.crs), 
                                            how = 'left')
yg_val_footprints = yg_val_footprints.rename(columns={"index_right": "bbox_id"})
yg_val_footprints['dataset'] = 'yg_val'
yg_val_footprints

nitze_footprints = nitze_footprints.sjoin(nitze_bboxes.to_crs(nitze_footprints.crs), 
                                            how = 'left')
nitze_footprints = nitze_footprints.rename(columns={"index_right": "bbox_id"})
nitze_footprints['dataset'] = 'nitze'
nitze_footprints

In [ ]:
test = yg_val_footprints.merge(yg_val_images,
                               how = 'right',
                               on = [])

## Create Composites

In [ ]:
# create a composite image across all regions for just 2019 and for 2018 and 2019 combined
planet_2019 = planet.filter(ee.Filter.stringContains('ID', '2019'))

planet_composites_all = planet.median()
planet_composites_2019 = planet_2019.median()

## Map Composites

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet_composites_2019)

In [ ]:
# Add composites to the map as one layer
Map.addLayer(ee.ImageCollection(planet_composites_2019),
             vis_params_imagery,
             '2019 Composites')
Map.addLayer(ee.ImageCollection(planet_composites_all),
             vis_params_imagery,
             '2018 +2019 Composites')

In [ ]:
Map

## Export Annual Composites

In [ ]:
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/nitze_regions/bboxes/nitze_bbox_water_removed.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
aoi['region'] = list(range(0, 7))
# convert to json for planet data search
sites = json.loads(aoi.to_json()) # if multiple sites
aoi

In [ ]:
zones = pd.DataFrame(columns = ['region', 'utm_zone'])
for idx, region in enumerate(aoi.geometry):
    region_zones = []
    for x, y in zip(region.exterior.coords.xy[0], region.exterior.coords.xy[1]):
        region_zones.append(utm_from_wgs84(x, y))
        
    region_zones = round(statistics.median(region_zones))
    temp_df = pd.DataFrame({'region': [idx],
                            'utm_zone': [region_zones]})
    zones = pd.concat([zones, temp_df])
zones = zones.set_index('region')  
zones

In [ ]:
# Export Composites to Drive
for row in aoi.iterrows():
    pprint(row)
    region = row[1]['region']
    name = 'nitze_regions_' + str(region) + '_2019_composite'
    geometry = sites['features'][region]['geometry']['coordinates']
    crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
    print(crs)
    task = ee.batch.Export.image.toCloudStorage(
        image = planet_composites_2019,
        description = name,
        bucket = 'abrupt_thaw',
        fileNamePrefix = 'planet_processing/data/nitze_regions/calibrated_composites/' + name,
        crs = crs,
        region = geometry,
        scale = 3,
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF',
        formatOptions = {'cloudOptimized': True}
    )
    task.start()

In [ ]:
# Export Composites to Drive (composites include 2018 images)
for row in aoi.iterrows():
    pprint(row)
    region = row[1]['region']
    name = 'nitze_regions_' + str(region) + '_all_composite'
    geometry = sites['features'][region]['geometry']['coordinates']
    crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
    print(crs)
    task = ee.batch.Export.image.toCloudStorage(
        image = planet_composites_all,
        description = name,
        bucket = 'abrupt_thaw',
        fileNamePrefix = 'planet_processing/data/nitze_regions/calibrated_composites/' + name,
        crs = crs,
        region = geometry,
        scale = 3,
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF',
        formatOptions = {'cloudOptimized': True}
    )
    task.start()